In [3]:
import dns.resolver
import dns.message
import dns.query
import dns.rdatatype

def recursive_query(query_name, query_type):
    query = dns.message.make_query(query_name, query_type, want_dnssec=False)
    query.flags |= 1

    response = dns.query.udp(query, '8.8.8.8')

    print(f"\nQuery: {query_name} - Type: {query_type}")
    print("Answer Section:")
    for answer in response.answer:
        print(answer)

    is_authoritative = False
    if response.flags & dns.flags.AA:
        is_authoritative = True
        print("Response is from an authoritative name server.")
    else:
        print("Response is not from an authoritative name server.")

    return response, is_authoritative


def iterative_query(query_name, query_type):
    current_ns = '8.8.8.8'
    while True:
        query = dns.message.make_query(query_name, query_type, want_dnssec=False)
        query.flags |= 0

        response = dns.query.udp(query, current_ns)

        if response.answer:
            print(f"\nFinal Answer from {current_ns}:")
            for answer in response.answer:
                print(answer)
            break
        else:
            if response.additional:
                for rrset in response.additional:
                    if rrset.rdtype == dns.rdatatype.A:
                        current_ns = rrset[0].address
                        break
            else:
                ns_rrset = response.authority[0]
                ns_name = ns_rrset[0].target
                ns_response = dns.resolver.resolve(ns_name, 'A')
                current_ns = ns_response[0].address


# Example usage
if __name__ == "__main__":
    recursive_query('www.sina.com.cn', 'A')
    iterative_query('www.sina.com.cn', 'A')



Query: www.sina.com.cn - Type: A
Answer Section:
www.sina.com.cn. 60 IN CNAME spool.grid.sinaedge.com.
spool.grid.sinaedge.com. 60 IN A 121.194.5.14
Response is not from an authoritative name server.

Final Answer from 8.8.8.8:
www.sina.com.cn. 60 IN CNAME spool.grid.sinaedge.com.
spool.grid.sinaedge.com. 60 IN A 121.194.5.14
